table of contents
1. データ取得

In [1]:
import os

import urllib.request
from datetime import datetime

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# テーブル取得

In [2]:
df_list = list()
for year in [2015+i for i in range(6)]:
    for i in range(1,4):
        url = f'https://soccer-db.net/contents/{year}_j{i}_newcomers.php' if i >= 2 else f'https://soccer-db.net/contents/{year}_j_newcomers.php'
        data = urllib.request.urlopen(url)
        soup = BeautifulSoup(data, "html")

        if year <= 2016:
            player_name_list = [div.text.replace(' ', '') for div in soup.find_all('div', 'mini_jpn_title')]
        else:
            player_name_list = [span.text.replace(' ', '') for span in soup.find_all('span', 'gt_j')]

        team_name_list = [td.text for td in soup.find_all('td') if td.get('width')=='30%']
        df = pd.DataFrame(data={'player_name':player_name_list, 'from_team_name':team_name_list})
        df['年度'] = f'{year}年'; df['competition'] = f'J{i}リーグ'.translate(str.maketrans({chr(0x0021 + i): chr(0xFF01 + i) for i in range(94)}))
        df_list.append(df)
    
df = pd.concat(df_list)

In [3]:
df.head()

,player_name,from_team_name,年度,competition
0,茂木駿佑,仙台ユース,2015年,Ｊ１リーグ
1,西村拓真,富山第一高,2015年,Ｊ１リーグ
2,摂津颯登,山形ユース,2015年,Ｊ１リーグ
3,高木利弥,神奈川大,2015年,Ｊ１リーグ
4,大橋尚志,鹿島ユース,2015年,Ｊ１リーグ


In [4]:
df.to_csv(os.path.join('..', 'data', 'shinsotsu-player.csv'), index=False)